# IMPORT LIBRARY

In [1]:
import cv2
import numpy as np 
import os  
from matplotlib import pyplot as plt 
import time
import mediapipe as mp

# FUNCTION DEFINITION FOR OBJECT DETECTION

In [6]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #COLOR CONVERSION
    image.flags.writeable = False #image is no longer writeable
    results = model.process(image) #make prediction
    image.flags.writeable = True #image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #COLOR COVERSION
    return image, results

In [8]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [9]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121,22,76), thickness = 2, circle_radius= 4), mp_drawing.DrawingSpec(color=(121,44,250), thickness = 2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness = 2, circle_radius= 4), mp_drawing.DrawingSpec(color=(245,66,230), thickness = 2, circle_radius=2))

# TESTING THE OBJECT DETECTION

In [46]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic :
    while cap.isOpened():
        #Read Feed
        ret, frame = cap.read()
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        #draw landmarks
        draw_styled_landmarks(image, results)
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        key = cv2.waitKey(10)
        if key == 27 :
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# EXTRACTING KEYPOINTS FROM THE OBJECT LANDMARK

In [10]:
def extract_keypoint(results):
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([left_hand,right_hand])

# MAKING DIRECTORY FOR DATASETS

In [14]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello','thank you','i love you'])
no_sequences = 5
sequence_length = 5

In [15]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# COLLECTING DATA

In [16]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic :
    #loop through actions
    for action in actions:
        #loop through sequences/videos
        for sequence in range(no_sequences):
            #loop through video length
            for frame_num in range(sequence_length):
                
                #Read Feed
                ret, frame = cap.read()
                #make detection
                image, results = mediapipe_detection(frame, holistic)
                print(results)
                #draw landmarks
                draw_styled_landmarks(image, results)
                #apply wait logic
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,sequence), (15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                
                #New export keypoints
                keypoints = extract_keypoint(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                #Show to screen
                cv2.imshow('OpenCV Feed', image)
        
                #break gracefully
                key = cv2.waitKey(10)
                if key == 27 :
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [57]:
cap.release()
cv2.destroyAllWindows()

# PREPROCCESS DATA, CREATE LABELS, AND FEATURES

In [17]:
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from keras import utils, layers, models, callbacks
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'hello': 0, 'thank you': 1, 'i love you': 2}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [21]:
np.array(sequences).shape

(15, 5, 126)

In [22]:
np.array(labels).shape

(15,)

In [23]:
X = np.array(sequences)

In [24]:
y = to_categorical(labels).astype(int)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# DATASETS TRAINING

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [46]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [109]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape = (5,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(62, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [110]:
res = [0.7]

In [111]:
actions[np.argmax(res)]

'hello'

In [112]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [105]:
model.fit(X_train, y_train, epochs=5, callbacks=[tb_callback])

Epoch 1/5
1/1 [==============================] - 4s 4s/step - loss: 0.5353 - categorical_accuracy: 0.7857
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: 0.4992 - categorical_accuracy: 0.7857
Epoch 3/5
1/1 [==============================] - 0s 31ms/step - loss: 0.5211 - categorical_accuracy: 0.8571
Epoch 4/5
1/1 [==============================] - 0s 31ms/step - loss: 0.4691 - categorical_accuracy: 0.7857
Epoch 5/5
1/1 [==============================] - 0s 31ms/step - loss: 0.4873 - categorical_accuracy: 0.7857


In [106]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 5, 64)             48896     
                                                                 
 lstm_12 (LSTM)              (None, 5, 128)            98816     
                                                                 
 lstm_13 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 62)                4030      
                                                                 
 dense_10 (Dense)            (None, 32)                2016      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 203265 (794.00 KB)
Trainable params: 203

# MAKE PREDICTION

In [60]:
model.predict(X_test)

1/1 [==============================] - 0s 42ms/step


array([[1.7758413e-08, 9.9874818e-01, 1.2518312e-03]], dtype=float32)

In [96]:
actions[np.argmax(res[0])]

'hello'

In [97]:
actions[np.argmax(y_test[0])]

'thank you'

In [107]:
model.save('action.h5')

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [108]:
del model

In [113]:
model.load_weights('action.h5')

# EVALUATE THE MODEL USING CONFUSION MATRIX AND ACCURACY

In [52]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [100]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 718ms/step


In [101]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [102]:
ytrue

[1]

In [114]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 0],
        [0, 1]]], dtype=int64)

In [104]:
accuracy_score(ytrue, yhat)

1.0

# TEXT TO VOICE

In [70]:
from gtts import gTTS
from  pygame import mixer

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [71]:
def sound(text, language):
    output = gTTS(text=text, lang=language, slow=True)
    output.save(f'{text}.mp3')
    mixer.init()
    mixer.music.load(f'{text}.mp3')
    mixer.music.play()
    time.sleep(1)
    mixer.music.unload()
    os.remove(f'{text}.mp3')

# TEST IN REAL TIME

In [72]:
#new detection variables
sequence = []
sentence = []
temp= 0
threshold = 0.7

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic :
    while True:
        #Read Feed
        ret, frame = cap.read()
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        #draw landmarks
        draw_styled_landmarks(image, results)
        #prediction logic
        keypoints = extract_keypoint(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            temp += 1
            if (temp > 50):
                temp = 0
                sound(actions[np.argmax(res)], 'en')
            
        #visualization logic
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, actions[np.argmax(res)], (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        time.sleep(0.1)
        
        #break gracefully
        key = cv2.waitKey(10)
        if key == 27 :
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 1s 936ms/step
i love you
1/1 [==============================] - 0s 42ms/step
i love you
1/1 [==============================] - 0s 36ms/step
i love you
1/1 [==============================] - 0s 48ms/step
i love you
1/1 [==============================] - 0s 39ms/step
i love you
1/1 [==============================] - 0s 30ms/step
i love you
1/1 [==============================] - 0s 34ms/step
i love you
1/1 [==============================] - 0s 32ms/step
i love you
1/1 [==============================] - 0s 49ms/step
i love you
1/1 [==============================] - 0s 41ms/step
i love you
1/1 [==============================] - 0s 61ms/step
i love you
1/1 [==============================] - 0s 38ms/step
i love you
1/1 [==============================] - 0s 43ms/step
i love you
1/1 [==============================] - 0s 38ms/step
i love you
1/1 [==============================] - 0s 51ms/step
i love you
1/1 [==============================] - 0s 32ms/step
i 

In [36]:
cap.release()
cv2.destroyAllWindows()